In [ ]:
#New Call Function:
listof_IPs,size = get_info(filename)# two table calls one to get size and other to get list of IPs
number_of_peers = len(listof_IPs)
inc = size//number_of_peers
offset = 0
offset_lookup = {}
for i,IP in enumerate(listof_IPs):
    if i == number_of_peers-1:
        offset_lookup[IP] = size - inc*(number_of_peers - 1)
    else:
        offset_lookup[IP] = offset
        offset += inc
client_lookup = {}
for IP in listof_IPs:
    client_lookup[IP] = Client(IP,offset_lookup[IP],filename)


In [ ]:
#CALL FUNCTIONS
sourcefile = input("Enter Searchfile: ")
sourcefile += ",1"
everything = (input("Enter name of target file along with address directory as name,directory: "))
target,location = everything.split(',')
size,source = handshake(sourcefile)
receiving(target,source,location,size)

In [ ]:
#SERVER PROGRAM
import os
import socket 
import threading

class Server:
    
    def __init__(self,location):
        self.location = location
        os.chdir(self.location)#init
        self.HEADER = 64
        self.PORT = 5050
        self.SERVER = socket.gethostbyname(socket.gethostname())
        self.ADDR = (self.SERVER, self.PORT)
        self.FORMAT = 'utf-8'
        self.DISCONNECT_MESSAGE = "!DISCONNECT"
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server.bind(self.ADDR)
        self.start()

    def handle_client(self,conn,addr):
        print(f"[NEW CONNECTION] {addr} connected.")
        connected = True
        while connected:
            msg_length = conn.recv(self.HEADER).decode(self.FORMAT)
            if msg_length:
                msg_length = int(msg_length)
                msg = conn.recv(msg_length).decode(self.FORMAT)
                if msg == self.DISCONNECT_MESSAGE:
                    break
                msg,feat = msg.split(',')
                if feat == "1":
                    conn.send(f"{msg},{os.path.getsize(msg)}".encode(self.FORMAT))
                elif feat == "0":
                    sendfile(msg,conn)
        print(f"[DISCONNECTED] {addr}")
        conn.close()

    def chunks(self,msg,offset,chunksize):
        tempfile_path = 'TEMP\\temp'
        with open(msg,'rb') as file:
            with open(tempfile_path,'wb') as chunk:
                file.seek(offset)
                data = file.read(chunksize)
                chunk.write(data)
        return tempfile_path

    def sendfile(self,msg,conn):#make temp file and copy data acc to offset
        filename,start_offset,end_offset = msg.split(';')
        temp_offset = start_offset
        chunksize = min(end_offset - start_offset , 2048000)
        data_sent = 0
        tempfile = ''
        while data_sent != (end_offset - start_offset):
            tempfile = chunks(msg,temp_offset,chunksize)
            with open(tempfile,'rb') as file:
                data = file.read()
                conn.send(data)
                print(len(data))
            data_sent += chunksize
            temp_offset += chunksize
            chunksize = min(end_offset - data_sent , 2048000)
        conn.send(b'')        
        print("Sending complete")
    
    def start(self):
        print("[STARTING] server is starting...")
        self.server.listen()
        print(f"[LISTENING] Server is listening on {self.SERVER}")
        while True:
            conn, addr = self.server.accept()

            thread = threading.Thread(target=self.handle_client, args=(conn, addr))
            thread.start()

In [ ]:
server1 = Server("D:\\Shareable Folder")